In [ ]:
!git clone http://github.com/Zapy67/ATML-PA-4

Cloning into 'ATML-PA-4'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 47 (delta 20), reused 38 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 289.00 KiB | 10.32 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
!git pull http://github.com/Zapy67/ATML-PA-4

From http://github.com/Zapy67/ATML-PA-4
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


In [ ]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [ ]:
!ls

'ATML - PA 4.pdf'   fed_lib   README.md     task2.ipynb   task4.ipynb
 data		    LICENSE   task1.ipynb   task3.ipynb


#  FedSGD vs FedAvg on heterogeneous domains

In [1]:
import torch
import torch.nn as nn

c:\Users\shaya\anaconda3\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
import fed_lib
import importlib
importlib.reload(fed_lib)

<module 'fed_lib' from 'd:\\shayann\\Uni_Resources\\Semester_5\\CS-6304\\PA_4\\ATML-PA-4\\fed_lib\\__init__.py'>

In [3]:
!pip install fedlab

In [4]:
from fed_lib import fed_methods, fed_model, utils

19:45:55 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [86]:
trainset, testset = utils.get_cifar10()

In [91]:
train_subset, _  = torch.utils.data.random_split(trainset.dataset, [0.01, 0.99]) 

In [93]:
len(train_subset)

500

In [99]:
importlib.reload(utils)
importlib.reload(fed_model)
client_loaders = utils.get_heterogenous_domains(train_subset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [100]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.181 0.112 0.236 0.053 0.081 0.089 0.124 0.09  0.035]
Entropy of Label Distribution H(P(Y)): 1.975813537120855
Normalized Entropy of Label Distribution: 0.8992314929938615

Label Distribution (P(Y)): [0.018 0.055 0.041 0.028 0.128 0.096 0.147 0.159 0.083 0.245]
Entropy of Label Distribution H(P(Y)): 1.9839548697628098
Normalized Entropy of Label Distribution: 0.8616206522830728

Label Distribution (P(Y)): [0.21  0.099 0.084 0.133 0.097 0.038 0.023 0.035 0.137 0.143]
Entropy of Label Distribution H(P(Y)): 2.047720053594738
Normalized Entropy of Label Distribution: 0.8893135197588257

Label Distribution (P(Y)): [0.079 0.047 0.089 0.084 0.049 0.19  0.059 0.064 0.214 0.125]
Entropy of Label Distribution H(P(Y)): 2.0700619739807027
Normalized Entropy of Label Distribution: 0.899016492497572

Label Distribution (P(Y)): [0.007 0.182 0.045 0.197 0.144 0.088 0.147 0.153 0.036]
Entropy of Label Distribution H(P(Y)): 1.9066229668126904
Normalized Entropy of Label Dist

In [ ]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [78]:
client_weights

[0.17986, 0.25772, 0.20686, 0.19112, 0.16444]

In [105]:
partition = "dirichlet"
device = "cpu"
alpha = 0.1
fed = fed_model.Federation(5, None, partition, domains, alpha, device, batch_size=64, pin_memory=True, num_workers=2)

In [106]:
client_loaders = fed.client_dataloaders
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [107]:
fed.set_method(fed_methods.FedAvg(local_epochs=5, client_weights=client_weights))

In [108]:
fed.train(10, lr=0.05, verbose=True)


--- Round 1/10 ---
Training Clients
Selected 5/5 clients: [3, 4, 0, 1, 2]
Training Client 4 for 5 epochs


c:\Users\shaya\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  Client 4: trained on 10095 samples
Training Client 5 for 5 epochs
  Client 5: trained on 12909 samples
Training Client 1 for 5 epochs
  Client 1: trained on 10170 samples
Training Client 2 for 5 epochs
  Client 2: trained on 10867 samples
Training Client 3 for 5 epochs
  Client 3: trained on 5959 samples
Average client drift: 3.927013
Training Server
Aggregating 5 clients
Weights: ['0.200', '0.200', '0.200', '0.200', '0.200']
Training Central


Central avg grad norm: 1.4890
Evaluate on round 1:


d:\shayann\Uni_Resources\Semester_5\CS-6304\PA_4\ATML-PA-4\fed_lib\utils.py:538: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  relative_diff = l2_diff / (float(torch.norm(p1, p=2)) + 1e-10)


...

Central avg grad norm: 0.4526
Evaluate on round 9:


FedAvg  | Test Loss: 2.3286, Test Acc: 24.88%
Central | Test Loss: 2.2175, Test Acc: 18.37%
Param Difference: 1.105730e+01
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
task_head.1.weight                       6.533590e+00 8.292605e-01
backbone.1.block.0.weight                5.867511e+00 9.922091e-01
backbone.0.block.0.weight                5.202777e+00 7.623766e-01
backbone.1.block.1.weight                2.880806e+00 2.662437e-01
backbone.0.block.1.bias                  2.145238e+00 1.000246e+00
Overall Statistics:
Total L2 difference:        1.105730e+01
Total parameters:           20,234

--- Round 10/10 ---
Training Clients
Selected 5/5 clients: [3, 4, 0, 1, 2]
Training Client 4 for 5 epochs
  Client 4: trained on 10095 samples
Training Client 5 for 5 epochs
  Client 5: trained on 12909 samples
Training Client 1 for 5 epochs
  Client 1: trained on 10170 samples
Training Client 2 for 5 epoc

Central avg grad norm: 0.4182
Evaluate on round 10:


FedAvg  | Test Loss: 2.1934, Test Acc: 25.97%
Central | Test Loss: 2.2077, Test Acc: 19.18%
Param Difference: 1.172708e+01
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
task_head.1.weight                       6.923658e+00 8.374971e-01
backbone.1.block.0.weight                6.186155e+00 1.004980e+00
backbone.0.block.0.weight                5.504955e+00 7.745328e-01
backbone.1.block.1.weight                3.085975e+00 2.798716e-01
backbone.1.block.1.bias                  2.400999e+00 9.973469e-01
Overall Statistics:
Total L2 difference:        1.172708e+01
Total parameters:           20,234
Training Complete!


FedAvg  | Test Loss: 2.1934, Test Acc: 25.97%
Central | Test Loss: 2.2077, Test Acc: 19.18%
Param Difference: 1.172708e+01

Comparing Server vs Central

Top 5 layers with largest L2 differences:
----------------------------------------------------------------------
Layer                                         L2 Diff     Rel Diff
----------------------------------------------------------------------
task_head.1.weight                       6.923658e+00 8.374971e-01
backbone.1.block.0.weight                6.186155e+00 1.004980e+00
backbone.0.block.0.weight                5.504955e+00 7.745328e-01
backbone.1.block.1.weight                3.085975e+00 2.798716e-01
backbone.1.block.1.bias                  2.400999e+00 9.973469e-01

Overall Statistics:
Total L2 difference:        1.172708e+01
Total L-inf difference:     7.500830e-01
Avg relative difference:    5.819457e-01
Total parameters:           20,234



In [7]:
from fed_lib import utils

In [8]:
import copy
clients = [utils.SmallCNN() for i in range(5)]

In [57]:
param_groups = zip(*[list(local_model.parameters()) for local_model in clients])

In [53]:
aggregated_params = [torch.mean(torch.stack(param_group, dim=0)) for param_group in param_groups]

In [ ]:
import torch
for param_group in param_groups:
    print(torch.mean(torch.stack(param_group), dim=0).shape)

torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [60]:
test = utils.SmallCNN()
for p in test.parameters():
    print(p.shape)

torch.Size([32, 3, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
torch.Size([64])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [34]:
list(param_groups)

[]

tensor(0.0069, grad_fn=<MeanBackward0>)